In [2]:
import pandas as pd
import glob
import sys
import os

In [60]:
accession="GCA_000001405.29"
threshold_short=1000000
threshold_ends=0.1

In [53]:
new_directory = f"/lab/wengpj01/vertebrate_pipeline/subdirs/{accession}"
os.chdir(new_directory)

In [54]:
# Read chromsizes.csv into a pandas dataframe
chromsizes = pd.read_csv('chromsizes.csv', sep='\t', header=None, names=['chr', 'length'])

# Find and read all *_from_pipeline.gtf files into a list of dataframes
gtf_files = glob.glob('*_from_pipeline.gtf')
gene_dfs = []

for gtf_file in gtf_files:
    df = pd.read_csv(gtf_file, sep='\t', header=None, names=['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute'])
    gene_dfs.append(df)
    
    # Strip text before the last underscore in the 'seqname' column
    df['seqname'] = df['seqname'].str.replace(r'^.*_', '', regex=True)

# Concatenate all dataframes into a single dataframe
genes = pd.concat(gene_dfs, ignore_index=True)


In [55]:
# Iterate through rows of the 'genes' dataframe and find corresponding chromosome length
genes['chr_length'] = None  # Create an empty column to store chromosome lengths
genes['chr_mini'] = None
genes['ends'] = None

for i, gene_row in genes.iterrows():
    seqname = gene_row['seqname']
    
    # Find the corresponding row in 'chromsizes'
    chrom_row = chromsizes[chromsizes['chr'] == seqname]
    
    if not chrom_row.empty:
        chr_length = chrom_row.iloc[0]['length']
        genes.at[i, 'chr_length'] = chr_length
        
        to_end = (chr_length - gene_row["start"])/chr_length
        to_start = (gene_row["start"])/chr_length
        
        if (to_start < threshold_ends) or (to_end < threshold_ends):
            genes.at[i, 'ends'] = "True"
        else:
            genes.at[i, 'ends'] = "False"
        
        if chr_length < threshold_short:
            genes.at[i, 'chr_mini'] = "True"
        else:
            genes.at[i, 'chr_mini'] = "False"
            

In [56]:
# Calculate the fraction of rows where 'chr_mini' is True
fraction_chr_mini_true = len(genes[genes['chr_mini'] == "True"]) / len(genes)
print(fraction_chr_mini_true)

0.5333333333333333


In [57]:
# Calculate the fraction of rows where 'ends' is True
fraction_near_ends = len(genes[genes['ends'] == "True"]) / len(genes)
print(fraction_near_ends)

0.3333333333333333


In [58]:
# Calculate the fraction of rows where 'ends' is True and 'chr_mini' is False
fraction_near_ends_long = len(genes[(genes['ends'] == "True") & (genes['chr_mini'] == "False")]) / len(genes[genes['chr_mini'] == "False"])
print(fraction_near_ends_long)

0.5


In [59]:
genes

,seqname,source,feature,start,end,score,strand,frame,attribute,chr_length,chr_mini,ends
0,CM000667.2,Genome,exon,9629133,9630032,.,-,0,"gene_id ""KH9511""; transcript_id ""KH9511.1""",181538259,False,True
1,CM000669.2,Genome,exon,122994759,122995634,.,-,0,"gene_id ""KH8006""; transcript_id ""KH8006.1""",159345973,False,False
2,CM000669.2,Genome,exon,141764159,141765109,.,+,0,"gene_id ""KH3992""; transcript_id ""KH3992.1""",159345973,False,False
3,CM000669.2,Genome,exon,141778489,141779388,.,+,0,"gene_id ""KH3483""; transcript_id ""KH3483.1""",159345973,False,False
4,CM000669.2,Genome,exon,141790101,141791261,.,+,0,"gene_id ""KH5175""; transcript_id ""KH5175.1""",159345973,False,False
5,CM000669.2,Genome,exon,141972688,141973689,.,-,0,"gene_id ""KH3818""; transcript_id ""KH3818.1""",159345973,False,False
6,CM000669.2,Genome,exon,143183419,143184435,.,+,0,"gene_id ""KH11253""; transcript_id ""KH11253.1""",159345973,False,False
7,CM000669.2,Genome,exon,143222079,143223050,.,+,0,"gene_id ""KH2887""; transcript_id ""KH2887.1""",159345973,False,False
8,CM000674.2,Genome,exon,10806051,10806980,.,-,0,"gene_id ""KH1732""; transcript_id ""KH1732.1""",133275309,False,True
9,CM000674.2,Genome,exon,10938254,10939222,.,-,0,"gene_id ""KH11772""; transcript_id ""KH11772.1""",133275309,False,True
